In [14]:
import ray
from ray import tune,air
from ray.rllib.algorithms.ppo import PPOConfig

def main():
    ray.init(ignore_reinit_error=True, log_to_driver=False)
    
    config = {
        "env": 'InvertedPendulum-v4',
        "framework": "torch",  # You can also use "tf" for TensorFlow
        "num_workers": 8,      # Number of parallel workers for collecting experiences
        "num_gpus": 0,         # Number of GPUs to use
        "train_batch_size": 4000,
        "model": {
            "use_lstm": False,
        },
    }

    tuner = tune.Tuner(
        "PPO",
        run_config=air.RunConfig(
            stop={"training_iteration": 50},
        ),
        param_space=config,
    )

    results = tuner.fit()

    ray.shutdown()
    return results


In [15]:
results = main()

2023-08-11 23:55:59,896	INFO worker.py:1474 -- Calling ray.init() again after it has already been called.


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
PPO_InvertedPendulum-v4_90e48_00000,200000,"{'ObsPreprocessorConnector_ms': 0.0035986900329589844, 'StateBufferConnector_ms': 0.0018749237060546875, 'ViewRequirementAgentConnector_ms': 0.055733680725097656}","{'num_env_steps_sampled': 200000, 'num_env_steps_trained': 200000, 'num_agent_steps_sampled': 200000, 'num_agent_steps_trained': 200000}",{},2023-08-11_23-58-44,True,986.01,{},1000,986.01,433,5,2543,michaelmbp.local,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 2.824068348958928, 'cur_kl_coeff': 0.04218750000000001, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 0.0029278307318529716, 'policy_loss': 0.0023690313952023623, 'vf_loss': 5.903852319710084e-07, 'vf_explained_var': -0.9924795518639267, 'kl': 0.01323160667682446, 'entropy': 0.0039128289663142735, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 46035.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 200000, 'num_env_steps_trained': 200000, 'num_agent_steps_sampled': 200000, 'num_agent_steps_trained': 200000}",50,127.0.0.1,200000,200000,200000,4000,1316.78,200000,4000,1316.78,0,8,0,0,4000,"{'cpu_util_percent': 51.099999999999994, 'ram_util_percent': 71.42500000000001}",58744,{},{},{},"{'mean_raw_obs_processing_ms': 0.1723045486893441, 'mean_inference_ms': 0.49765163906935383, 'mean_action_processing_ms': 0.09231205756461987, 'mean_env_wait_ms': 0.05119145226531757, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 1000.0, 'episode_reward_min': 433.0, 'episode_reward_mean': 986.01, 'episode_len_mean': 986.01, 'episode_media': {}, 'episodes_this_iter': 5, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 433.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 652.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 516.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0], 'episode_lengths': [1000, 1000, 1000, 1000, 1000, 433, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 652, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 516,

2023-08-11 23:58:44,586	INFO tune.py:1111 -- Total run time: 164.69 seconds (164.55 seconds for the tuning loop).


ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 2.824068348958928, 'cur_kl_coeff': 0.04218750000000001, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 0.0029278307318529716, 'policy_loss': 0.0023690313952023623, 'vf_loss': 5.903852319710084e-07, 'vf_explained_var': -0.9924795518639267, 'kl': 0.01323160667682446, 'entropy': 0.0039128289663142735, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 46035.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 200000, 'num_env_steps_trained': 200000, 'num_agent_steps_sampled': 200000, 'num_agent_steps_trained': 200000}, 'sampler_results': {'episode_reward_max': 1000.0, 'episode_reward_min': 433.0, 'episode_reward_mean': 986.01, 'episode_len_mean': 986.01, 'episode_media': {}, 'episodes_this_iter': 5, 'policy_reward_min'

In [ ]:
# Get the best result
r=results.get_best_result()

# Get the best checkpoint
cp=r.checkpoint

In [17]:
from ray.rllib.algorithms.algorithm import Algorithm

algo=Algorithm.from_checkpoint(cp)

2023-08-12 00:01:09,444	WARNING algorithm_config.py:643 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-08-12 00:01:09,448	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-08-12 00:01:11,309	INFO worker.py:1636 -- Started a local Ray instance.
(RolloutWorker pid=59227) /Users/spacefarers/mambaforge/envs/rl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(RolloutWorker pid=59227)   if not isinstance(terminated, (bool, np.bool8)):


In [26]:
import time
env=gym.make("InvertedPendulum-v4")
for episode in range(100):
    obs,info = env.reset()
    print(obs)
    done = False
    while not done:
        action = algo.compute_single_action(obs)
        print(action)
        obs,reward,done,info,ads=env.step(action)
        env.render()
        time.sleep(0.1)

[-0.00554944  0.00480566  0.00666359  0.00033744]
[-0.09312105]
[0.18475556]


/Users/spacefarers/mambaforge/envs/rl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


[1.1251249]
[-3.]
[3.]
[-2.3875623]
[2.9713182]
[-3.]
[3.]
[-3.]
[0.67960596]
[1.2587986]
[-2.1491108]
[3.]
[-2.4104638]
[1.2387915]
[-0.73907804]
[1.3493633]
[-1.9549251]
[2.7799497]
[-3.]
[2.8383408]
[-3.]
[2.3665624]
[-2.821508]
[3.]
[-3.]
[3.]
[-3.]
[3.]
[-1.9225649]
[1.573844]
[-2.1329708]
[2.4514399]
[-3.]
[3.]
[-1.1881506]
[-1.2811904]
[3.]
[-3.]
[2.6658864]
[-3.]
[2.7973828]
[-2.6217477]
[2.6803637]
[-2.701475]
[3.]
[-3.]
[3.]
[-3.]
[2.7514296]
[-3.]
[3.]
[-2.045198]
[0.7334151]
[-2.2520266]
[3.]
[-1.8903923]
[3.]
[-3.]
[3.]
[-3.]
[1.5697365]
[-2.9003384]
[3.]
[-2.2192879]


KeyboardInterrupt: 